In [ ]:
!pip install tf-nightly

In [3]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
import os
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 2578, done.
remote: Counting objects: 100% (2578/2578), done.
remote: Compressing objects: 100% (2149/2149), done.
remote: Total 2578 (delta 641), reused 1273 (delta 397), pack-reused 0
Receiving objects: 100% (2578/2578), 32.51 MiB | 35.45 MiB/s, done.
Resolving deltas: 100% (641/641), done.


In [ ]:
# Install the Object Detection API
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .


Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-cp37-none-any.whl size=1643278 sha256=0047ee7cba46891b373265ddf6c77d42332eaa3de1c5c6dad62a416270e8d818
  Stored in directory: /tmp/pip-ephem-wheel-cache-ymm9zy4m/wheels/94/49/4b/39b051683087a22ef7e80ec52152a27249d1a644ccf4e442ea
  Created wheel for avro-python3: filename=avro_python3-1.10.2-cp37-none-any.whl size=44011 sha256=700c44abe2dc586ea072d1b436fef24de7d9a71db5f09b1aba917be0bdcfc587
  Stored in directory: /root/.cache/pip/wheels/ee/ee/18/c466221ca6900e3efce2f4ea9c329288808679aecdcb2838d3
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491058 sha256=ee534d732f83dd6e4794a1a2409f153aa003ee909e636b8014f17ad22e239664
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for dill: filename=dill-0.3.1.1-cp37-none-any.whl size=78532 sha256=771506d0c226bede62019a540e8e13d220f38c25e5a9c0e960be

ERROR: multiprocess 0.70.11.1 has requirement dill>=0.3.3, but you'll have dill 0.3.1.1 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: apache-beam 2.28.0 has requirement avro-python3!=1.9.2,<1.10.0,>=1.8.1, but you'll have avro-python3 1.10.2 which is incompatible.


In [ ]:
!mkdir /content/drive/MyDrive/SignLanguageMobilenetV2

In [4]:
#if the below link doesn't work Generate the data set from here 
# https://app.roboflow.com/dataset/american-sign-language-letters-14kx4/generate
!curl -L "https://app.roboflow.com/ds/Xm8xzbRAxk?key=ZMSy423z9F" > /content/drive/MyDrive/SignLanguageMobilenetV2/roboflow.zip;

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   891  100   891    0     0   1143      0 --:--:-- --:--:-- --:--:--  1142
100 10.3M  100 10.3M    0     0  8921k      0  0:00:01  0:00:01 --:--:-- 8921k


In [5]:
!mkdir /content/drive/MyDrive/SignLanguageMobilenetV2/data
!unzip /content/drive/MyDrive/SignLanguageMobilenetV2/roboflow.zip -d /content/drive/MyDrive/SignLanguageMobilenetV2/data

mkdir: cannot create directory ‘/content/drive/MyDrive/SignLanguageMobilenetV2/data’: File exists
Archive:  /content/drive/MyDrive/SignLanguageMobilenetV2/roboflow.zip
replace /content/drive/MyDrive/SignLanguageMobilenetV2/data/README.roboflow.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: Y
 extracting: /content/drive/MyDrive/SignLanguageMobilenetV2/data/README.roboflow.txt  
replace /content/drive/MyDrive/SignLanguageMobilenetV2/data/test/Letters.tfrecord? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
 extracting: /content/drive/MyDrive/SignLanguageMobilenetV2/data/test/Letters.tfrecord  
 extracting: /content/drive/MyDrive/SignLanguageMobilenetV2/data/test/Letters_label_map.pbtxt  
 extracting: /content/drive/MyDrive/SignLanguageMobilenetV2/data/train/Letters.tfrecord  
 extracting: /content/drive/MyDrive/SignLanguageMobilenetV2/data/train/Letters_label_map.pbtxt  


In [ ]:
Y# NOTE: Update these TFRecord names from "cells" and "cells_label_map" to your files!
test_record_fname = '/content/drive/MyDrive/SignLanguageMobilenetV2/data/valid/Letters.tfrecord'
train_record_fname = '/content/drive/MyDrive/SignLanguageMobilenetV2/data/train/Letters.tfrecord'
label_map_pbtxt_fname = '/content/drive/MyDrive/SignLanguageMobilenetV2/data/train/Letters_label_map.pbtxt'

In [ ]:
##change chosen model to deploy different models available in the TF2 object detection zoo
MODELS_CONFIG = {
    'ssd-mobilenet' : {
        'model_name': 'ssd_mobilenet_v2',
        'base_pipeline_file': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz',
        'batch_size' : 16
    },
    'efficientdet-d0': {
        'model_name': 'efficientdet_d0_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d0_512x512_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d0_coco17_tpu-32.tar.gz',
        'batch_size': 16
    },
    'efficientdet-d1': {
        'model_name': 'efficientdet_d1_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d1_640x640_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d1_coco17_tpu-32.tar.gz',
        'batch_size': 16
    },
    'efficientdet-d2': {
        'model_name': 'efficientdet_d2_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d2_768x768_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d2_coco17_tpu-32.tar.gz',
        'batch_size': 16
    },
        'efficientdet-d3': {
        'model_name': 'efficientdet_d3_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d3_896x896_coco17_tpu-32.config',
        'pretrained_checkpoint': 'efficientdet_d3_coco17_tpu-32.tar.gz',
        'batch_size': 16
    }
}

#in this tutorial we implement the lightweight, smallest state of the art efficientdet model
#if you want to scale up tot larger efficientdet models you will likely need more compute!
chosen_model = 'ssd-mobilenet'

num_steps = 40000 #The more steps, the longer the training. Increase if your loss function is still decreasing and validation metrics are increasing. 
num_eval_steps = 500 #Perform evaluation after so many steps

model_name = MODELS_CONFIG[chosen_model]['model_name']
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']
base_pipeline_file = MODELS_CONFIG[chosen_model]['base_pipeline_file']
batch_size = MODELS_CONFIG[chosen_model]['batch_size'] #if you can fit a large batch in memory, it may speed up your training

In [ ]:
#download pretrained weights
%cd /content/drive/MyDrive/SignLanguageMobilenetV2
import tarfile
download_tar = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/' + pretrained_checkpoint

!wget {download_tar}
tar = tarfile.open(pretrained_checkpoint)
tar.extractall()
tar.close()

/content/drive/MyDrive/SignLanguageMobilenetV2
--2021-04-07 06:59:59--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.217.15.80, 2607:f8b0:4004:810::2010
Connecting to download.tensorflow.org (download.tensorflow.org)|172.217.15.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20515344 (20M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’

ssd_mobilenet_v2_fp 100%[===================>]  19.56M  89.3MB/s    in 0.2s    

2021-04-07 07:00:00 (89.3 MB/s) - ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’ saved [20515344/20515344]



In [ ]:
#download base training configuration file
%cd /content/drive/MyDrive/SignLanguageMobilenetV2
download_config = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/' + base_pipeline_file
!wget {download_config}

/content/drive/MyDrive/SignLanguageMobilenetV2
--2021-04-07 07:01:56--  https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4684 (4.6K) [text/plain]
Saving to: ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config’

ssd_mobilenet_v2_fp 100%[===================>]   4.57K  --.-KB/s    in 0.001s  

2021-04-07 07:01:56 (5.73 MB/s) - ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config’ saved [4684/4684]



In [ ]:
#prepare
pipeline_fname = '/content/drive/MyDrive/SignLanguageMobilenetV2/' + base_pipeline_file
fine_tune_checkpoint = '/content/drive/MyDrive/SignLanguageMobilenetV2/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'

def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())
num_classes = get_num_classes(label_map_pbtxt_fname)

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [ ]:
#write custom configuration file by slotting our dataset, model checkpoint, and training parameters into the base pipeline file

import re

%cd /content/drive/MyDrive/SignLanguageMobilenetV2
print('writing custom configuration file')

with open(pipeline_fname) as f:
    s = f.read()
with open('pipeline_file.config', 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    
    #fine-tune checkpoint type
    s = re.sub(
        'fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "{}"'.format('detection'), s)
        
    f.write(s)

/content/drive/MyDrive/SignLanguageMobilenetV2
writing custom configuration file


In [ ]:
%cat /content/drive/MyDrive/SignLanguageMobilenetV2/pipeline_file.config

# SSD with Mobilenet v2 FPN-lite (go/fpn-lite) feature extractor, shared box
# predictor and focal loss (a mobile version of Retinanet).
# Retinanet: see Lin et al, https://arxiv.org/abs/1708.02002
# Trained on COCO, initialized from Imagenet classification checkpoint
# Train on TPU-8
#
# Achieves 22.2 mAP on COCO17 Val

model {
  ssd {
    inplace_batchnorm_update: true
    freeze_batchnorm: false
    num_classes: 26
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
        use_matmul_gather: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    encode_background_as_zeros: true
    anchor_generator {
      multiscale_anchor_generator 

In [ ]:
pipeline_file = '/content/drive/MyDrive/SignLanguageMobilenetV2/pipeline_file.config'
model_dir = '/content/drive/MyDrive/SignLanguageMobilenetV2/training'

In [ ]:
!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_file} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1 \
    --num_eval_steps={num_eval_steps}

Traceback (most recent call last):
  File "/content/models/research/object_detection/model_main_tf2.py", line 113, in <module>
    tf.compat.v1.app.run()
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/platform/app.py", line 40, in run
    _run(main=main, argv=argv, flags_parser=_parse_flags_tolerate_undef)
  File "/usr/local/lib/python3.7/dist-packages/absl/app.py", line 303, in run
    _run_main(main, args)
  File "/usr/local/lib/python3.7/dist-packages/absl/app.py", line 251, in _run_main
    sys.exit(main(argv))
  File "/content/models/research/object_detection/model_main_tf2.py", line 110, in main
    record_summaries=FLAGS.record_summaries)
  File "/usr/local/lib/python3.7/dist-packages/object_detection/model_lib_v2.py", line 667, in train_loop
    loss = _dist_train_step(train_input_iter)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/eager/def_function.py", line 828, in __call__
    result = self._call(*args, **kwds)
  File "/usr/local/lib/pyt

In [ ]:
#run model evaluation to obtain performance metrics
#!python /content/models/research/object_detection/model_main_tf2.py \
    #--pipeline_config_path={pipeline_file} \d
    #--model_dir={model_dir} \
    #--checkpoint_dir={model_dir} \
#Not yet implemented for EfficientDet

## Exporting a Trained Inference Graph
Still to come for TF2 models, we will be updating this Colab notebook accordingly as the functionality is added. 

In [ ]:
#see where our model saved weights
%ls '/content/drive/MyDrive/SignLanguageMobilenetV2/training'

checkpoint  ckpt-1.data-00000-of-00001  ckpt-1.index  train/


In [ ]:
#run conversion script
import re
import numpy as np

output_directory = '/content/drive/MyDrive/SignLanguageMobilenetV2/fine_tuned_model'

#place the model weights you would like to export here
last_model_path = '/content/drive/MyDrive/SignLanguageMobilenetV2/training'
print(last_model_path)
!python /content/models/research/object_detection/exporter_main_v2.py \
    --trained_checkpoint_dir {last_model_path} \
    --output_directory {output_directory} \
    --pipeline_config_path {pipeline_file}

/content/drive/MyDrive/SignLanguageMobilenetV2/training
2021-04-07 13:28:54.747570: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-04-07 13:28:56.823630: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-07 13:28:56.824499: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-04-07 13:28:56.856433: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-04-07 13:28:56.857013: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2021-04-07 13:28:56.857051:

In [ ]:
%ls '/content/drive/MyDrive/SignLanguageMobilenetV2/fine_tuned_model/saved_model/'

assets/  saved_model.pb  variables/


In [ ]:
%ls '/content/drive/MyDrive/SignLanguageMobilenetV2/training/'

checkpoint  ckpt-1.data-00000-of-00001  ckpt-1.index  train/


In [ ]:
#exporting 
!python /content/models/research/object_detection/export_tflite_graph_tf2.py \
 --pipeline_config_path {pipeline_file} \
 --trained_checkpoint_dir {last_model_path} \
 --output_directory /content/drive/MyDrive/SignLanguageMobilenetV2/output/exported_models/tflite_inference

2021-04-07 13:34:54.863140: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-04-07 13:34:56.602901: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-04-07 13:34:56.638179: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-04-07 13:34:56.639032: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1734] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2021-04-07 13:34:56.639088: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-04-07 13:34:56.643283: I tensorflow/stream_executor/platform/default

In [ ]:
saved_tflite_inference = "/content/drive/MyDrive/SignLanguageMobilenetV2/output/exported_models/tflite_inference/saved_model"
/content/drive/MyDrive/SignLanguageMobilenetV2/output/exported_models/tflite_inference/saved_model

In [ ]:
def representative_dataset():
    for _ in range(100):
      data = np.random.rand(1, 320, 320, 3)
      yield [data.astype(np.float32)]

In [ ]:
import numpy as np
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_saved_model(saved_tflite_inference)
converter.allow_custom_ops = True
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.inference_input_type = tf.uint8  # or tf.uint8
converter.inference_output_type = tf.uint8  # or tf.uint8
tflite_quant_model = converter.convert()

In [ ]:
tf_lite_model_path_without_metadata = "/content/drive/MyDrive/SignLanguageMobilenetV2/output/tflite/detect_without_metadata.tflite"

In [ ]:
with tf.io.gfile.GFile(tf_lite_model_path_without_metadata, 'wb') as f:
  f.write(tflite_quant_model)

NotFoundError: ignored

## Testing the Input and Output of model

In [ ]:
import tensorflow as tf

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=tf_lite_model_path_without_metadata)

print(interpreter.get_input_details()[0]['shape'])
print(interpreter.get_input_details()[0]['dtype'])

# Print output shape and type
print(interpreter.get_output_details()[0]['shape'])
print(interpreter.get_output_details()[0]['dtype'])

## Writing Metadata to model for android/ios deployment

In [ ]:
''' Writing MetaData to TfLite Model '''

from tflite_support.metadata_writers import object_detector
from tflite_support.metadata_writers import writer_utils
from tflite_support import metadata

ObjectDetectorWriter = object_detector.MetadataWriter
_MODEL_PATH = tf_lite_model_path_without_metadata
_LABEL_FILE = "/content/drive/MyDrive/SignLanguageMobilenetV2/data/train/Letters_label_map.pbtxt"
_SAVE_TO_PATH = "/content/drive/MyDrive/SignLanguageMobilenetV2/output/tflite/detect_with_metadata.tflite"

writer = ObjectDetectorWriter.create_for_inference(
    writer_utils.load_file(_MODEL_PATH), [127.5], [127.5], [_LABEL_FILE])
writer_utils.save_file(writer.populate(), _SAVE_TO_PATH)

# Verify the populated metadata and associated files.
displayer = metadata.MetadataDisplayer.with_model_file(_SAVE_TO_PATH)
print("Metadata populated:")
print(displayer.get_metadata_json())
print("Associated file(s) populated:")
print(displayer.get_packed_associated_file_list())